In [30]:
#import dependencies
import requests
from citipy import citipy
import pandas as pd
import csv
import pandas as pd
import matplotlib.pyplot as plt
import json
import gmaps
from pprint import pprint
import matplotlib.pyplot as plt
weather_data_file = '../weatherpy/Global Weather Data.csv'
cities_with_data = '../weatherpy/weatherpy_city_list.csv'
#import api key
from vacation_config import g_api

In [3]:
weather_data_df = pd.read_csv(weather_data_file)
display(weather_data_df)

,City,Longitude,Latitude,Temperature,Wind Speed,Humidity,Cloudiness
0,Kutum,24.6667,14.2000,82.78,12.82,11,36
1,Lebu,-73.6500,-37.6167,52.18,8.01,68,73
2,Khatanga,102.5000,71.9667,37.85,9.82,78,99
3,Ilhabela,-45.3581,-23.7781,79.74,4.88,64,79
4,Port Elizabeth,25.5701,-33.9180,55.69,12.66,94,90
...,...,...,...,...,...,...,...
651,Piaçabuçu,-36.4344,-10.4056,76.37,7.67,78,67
652,Saldanha,17.9442,-33.0117,53.65,8.01,83,12
653,Maningrida,134.2667,-12.0667,68.22,8.16,91,66
654,Belmonte,-38.8828,-15.8631,79.30,4.85,81,100


In [4]:
#store location coordinates and humidity as floats for heat map
locations=weather_data_df[['Latitude','Longitude']].astype(float)
humidity=weather_data_df['Humidity'].astype(float)

In [10]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
#my perfect vacation parameters: Between 75-85 degrees F, low clouds, and wind speed under 10 mph
#drop by temperature bounds
vacation_df = weather_data_df[weather_data_df['Temperature']>=75]
vacation_df = vacation_df[vacation_df['Temperature']<=85]
#drop by wind speed
vacation_df = vacation_df[vacation_df['Wind Speed']<=10]
#drop by cloudiness
vacation_df = vacation_df[vacation_df['Cloudiness']<=15]
display(vacation_df)

,City,Longitude,Latitude,Temperature,Wind Speed,Humidity,Cloudiness
23,Laguna,-121.4238,38.4210,80.06,8.99,42,1
32,Atuona,-139.0333,-9.8000,78.89,9.46,72,12
44,Vila Velha,-40.2925,-20.3297,82.35,8.05,69,0
112,Portel,-50.8211,-1.9356,81.54,3.62,76,15
139,Ukiah,-123.2078,39.1502,81.00,5.75,33,1
239,Arraial do Cabo,-42.0278,-22.9661,80.46,8.05,74,0
268,Khuzdar,66.6434,27.7384,81.01,6.42,12,0
297,Nuevo Progreso,-97.9500,23.1667,83.73,8.01,64,9
341,Kano,8.5167,12.0001,80.49,2.30,74,0
358,Simdega,84.5167,22.6167,80.15,4.50,73,15


In [26]:
#add in hotel name and coordinates columns
vacation_df['Hotel Name']=""
vacation_df['Hotel Lat']=""
vacation_df['Hotel Lng']=""
vacation_df['Hotel City']=""
vacation_df["Hotel Country"]=""
display(vacation_df)

,City,Longitude,Latitude,Temperature,Wind Speed,Humidity,Cloudiness,Hotel Name,Hotel Lat,Hotel Lng,Hotel City,Hotel Country
23,Laguna,-121.4238,38.4210,80.06,8.99,42,1,,,,,
32,Atuona,-139.0333,-9.8000,78.89,9.46,72,12,,,,,
44,Vila Velha,-40.2925,-20.3297,82.35,8.05,69,0,,,,,
112,Portel,-50.8211,-1.9356,81.54,3.62,76,15,,,,,
139,Ukiah,-123.2078,39.1502,81.00,5.75,33,1,,,,,
239,Arraial do Cabo,-42.0278,-22.9661,80.46,8.05,74,0,,,,,
268,Khuzdar,66.6434,27.7384,81.01,6.42,12,0,,,,,
297,Nuevo Progreso,-97.9500,23.1667,83.73,8.01,64,9,,,,,
341,Kano,8.5167,12.0001,80.49,2.30,74,0,,,,,
358,Simdega,84.5167,22.6167,80.15,4.50,73,15,,,,,


In [28]:
#Use places call for hotels around the vacation spots
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "rankby": "distance",
    "types":"lodging",
    "keyword":"hotel",
    "key": g_api}

for index, row in vacation_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    params['location'] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    print(json.dumps(hotels, indent=4, sort_keys=True))
    
    try:
        vacation_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
        vacation_df.loc[index, "Hotel Lat"]=hotels["results"][0]["geometry"]["location"]["lat"]
        vacation_df.loc[index, "Hotel Lng"]=hotels["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


{
    "html_attributions": [],
    "next_page_token": "ATtYBwIRvKNh4-nbkWN0WbmwWjcrldiqeolGKXzancAaIH8js354gezxgpkhnPCrX3-H7pnKcGOiV4Lbvz4nV-Ip2GLQ37UF-yjkYGwomKnHFhg6s6mp7o9k4Y_9Wr31p7kqF9I4aLZYlfOSQgCQz3HQ1MeRPLD3CSo4hO7LUbnrLOdP7gcBjpgqA5tFyv--iIvNAp0udsjkC6LnQIxQ8gWkts_lxy6BPM2GJHrWEvkHZHGlDlGDOtyN_TrWP99wiCBGksWGLK773jMmooNIzipPzlNycjQGDi78r0SJYg0HPdmGS0Nus-z_KD9JpY9xjZb0U6jo58H6tOA_FmuvNDJCE87tZfCTMtDnO1jKy_y3TCVUVub_cFQpChQqTe3tqDwepNFSTRX-enAfRpVEMNLCbBmQ04J3tVKI8Umx8Dp5Cj67dGjiEYa6j8bZqXkkwjLF4HQQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.4240827,
                    "lng": -121.3987067
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.42550532989272,
                        "lng": -121.3974206701073
                    },
                    "southwest": {
                        "lat": 38.42

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.8008674,
                    "lng": -139.0324028
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.799511920107278,
                        "lng": -139.0313023201072
                    },
                    "southwest": {
                        "lat": -9.802211579892722,
                        "lng": -139.0340019798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Pension Temetiu Village",
            "photos": [
                {
                    "height": 3912,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/100753586291945627781\">

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL1BdACSPEED4tTng81Ivoo8JbiHmD_HwQBpLuF1KBwDm_8Sf8VqoT7ZfIHQqDvlFBH1maClFXg3zKUn9zKLzZDeanl8v1CiPFCnrZhdb_797G9VPd17aOBL4LRW02_J78OPX8j0pG8GiFvn4feaBJrl05zisOooC7kbn8xVYxSVyUjCa0OZcHxrlJq_4NfynzHb5onPmF3-PDLZTT2SQQNR9NlMA0E9FqECQ26cAG1bdQ8xyRtXq7_2rnP9v5Ws5bNWO2UczH723N9SirdmtNzffSjBHVQMrWyn7HdKxhjjUyDOYPN1SGWwDm2qkhsk4cTiGFENSz9OAAzX1QfW1f85p8WDfXbCUfpbyjKrPeCZ0tqPWa9s7nQx1MqbIt0O890C0AXhQlvcbp_HhGGCduSALhPaqmGBhC9DNocBNLzbyePigt6i0m6GZMfaGIk_UeH",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.3310853,
                    "lng": -40.2917534
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.32973077010728,
                        "lng": -40.29049272010728
                    },
                    "southwest": {
                        "lat": -20.

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -1.9381914,
                    "lng": -50.8233611
                },
                "viewport": {
                    "northeast": {
                        "lat": -1.936822770107278,
                        "lng": -50.82196542010728
                    },
                    "southwest": {
                        "lat": -1.939522429892723,
                        "lng": -50.82466507989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Pousada Serve Bem",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 618,
                    "html_attributions": [
                        "

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIlP4HYtuRWf3RJXUxlKRfSNMKsbj507YPY_krMh2jiutJ3liVxmnow2ITXrgYnoFunf1TfmRszupL4AGqGrn3ZGK9U-zxwCg2fDqzoBqB6gLp9zZsmNLKme7XYpnJ_UFlNorHpJJaJ5pzCPFIGh3pmLwgB7P76P8XwSEypq6inb5Gu5W1J89WuPawF5CdLBb1bF_trAgifzgLuOiB2OVCWVZaPILcR73cLe4smJwnOM2090PAG7Flclbvxc-FaagelSH6bLOTy3eCuk8_VddZCkk08gViZiNkNEfjnCDVOHHzyeeS23otnIXH2BJYKURaD2tg9qY0S4aPANra_GZv9--OCTqX6nh72IAq0Cq8OFF5Jlz8-M-hspBZPMkiFJvYQTJdvWUpLNGFJvya2z0VZuDvEFdg4_0NGyxTgy5K68UPET083K6_Q6g8zzcGkWolR",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.9670764,
                    "lng": -42.02745729999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.96568007010728,
                        "lng": -42.02607177010727
                    },
                    "southwest": {
                        "lat

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.8055171,
                    "lng": 66.61301360000002
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.80683572989272,
                        "lng": 66.61431852989273
                    },
                    "southwest": {
                        "lat": 27.80413607010728,
                        "lng": 66.61161887010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Al Mustafa Hotel",
            "photos": [
                {
                    "height": 3120,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/104992513327248788155\">taimur

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKQ6a-JFDGMbvqrPyIHJ0InxO2VaEbZZbl541ccPmGiunZHwxUpiCupvCm5uSn8dpqVBQKrU4oN1L1WhFA9Z-W0oQ2tOoa8vpo4u3Rg7RdhNxDJfdH2shju6uVGztqtNRXVlzxMrDLPiEJxPEngWpbETqBLRTYDO1PbsokDXVvDLU06VmRz1uUMPihFxpBlM5GwfxZVLuGqEthNCT9jf6dmK-qh03nNdEBSgNDd8j7o0vvimyGYHiNyL2QYSU96TtaB2pNOukUKJBe9f4PJyVEejjGKpc6i4WHk4ZmjVqQn30-oZ8A2sm_WzXPjPagmlJIBgJUUfO9hc7jrKCP90RRQqu6gUyvftyAQWK0qw3DiSDXjqHc1OGo79iPC-hi8zeB2kw76Kw5z7seKkcFyFy4R3LXvuEWUz4z_z1etqlQRZJFw_3S5upp6U5RGAVtMeSIO",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.6111706,
                    "lng": 84.50363139999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.61260347989272,
                        "lng": 84.50500512989271
                    },
                    "southwest": {
                        "lat": 2

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 21.3425294,
                    "lng": -158.1239544
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.34396077989272,
                        "lng": -158.1226885201073
                    },
                    "southwest": {
                        "lat": 21.34126112010728,
                        "lng": -158.1253881798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Ko Olina Resort",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 6656,
                    "html_attributions": [
                        "<a

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 7.525635100000001,
                    "lng": -12.500587
                },
                "viewport": {
                    "northeast": {
                        "lat": 7.527018629892722,
                        "lng": -12.49902952010728
                    },
                    "southwest": {
                        "lat": 7.524318970107277,
                        "lng": -12.50172917989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bonthe Holiday Village",
            "photos": [
                {
                    "height": 2304,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/113854676430808110740\

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 17.8814582,
                    "lng": -76.4065593
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.88254972989272,
                        "lng": -76.40544297010726
                    },
                    "southwest": {
                        "lat": 17.87985007010728,
                        "lng": -76.40814262989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Morant Villa",
            "photos": [
                {
                    "height": 3120,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/107066193770422067548\">Rogerlene Milw

In [29]:
display(vacation_df)

,City,Longitude,Latitude,Temperature,Wind Speed,Humidity,Cloudiness,Hotel Name,Hotel Lat,Hotel Lng,Hotel City,Hotel Country
23,Laguna,-121.4238,38.4210,80.06,8.99,42,1,Holiday Inn Express & Suites Elk Grove Central...,38.4241,-121.399,,
32,Atuona,-139.0333,-9.8000,78.89,9.46,72,12,Pension Temetiu Village,-9.80087,-139.032,,
44,Vila Velha,-40.2925,-20.3297,82.35,8.05,69,0,Hotel Prainha,-20.3311,-40.2918,,
112,Portel,-50.8211,-1.9356,81.54,3.62,76,15,Pousada Serve Bem,-1.93819,-50.8234,,
139,Ukiah,-123.2078,39.1502,81.00,5.75,33,1,Economy Inn Ukiah,39.1477,-123.208,,
239,Arraial do Cabo,-42.0278,-22.9661,80.46,8.05,74,0,Hotel Miramar,-22.9671,-42.0275,,
268,Khuzdar,66.6434,27.7384,81.01,6.42,12,0,Al Mustafa Hotel,27.8055,66.613,,
297,Nuevo Progreso,-97.9500,23.1667,83.73,8.01,64,9,Villas Del Tordo,23.0468,-97.7862,,
341,Kano,8.5167,12.0001,80.49,2.30,74,0,Royal Tropicana Hotel,12.0047,8.53644,,
358,Simdega,84.5167,22.6167,80.15,4.50,73,15,Mahabir Jain Bhawan,22.6112,84.5036,,


In [43]:
#use citipy to determine the city the hotels are in, then append city and country to dataframe
for index, row in vacation_df.iterrows():
    htl_lat = row['Hotel Lat']
    htl_lng = row['Hotel Lng']
    hotel_city = citipy.nearest_city(htl_lat, htl_lng).city_name
    hotel_country = citipy.nearest_city(htl_lat, htl_lng).country_code
    vacation_df.loc[index, "Hotel City"] = hotel_city
    vacation_df.loc[index, "Hotel Country"] = hotel_country

In [44]:
display(vacation_df)

,City,Longitude,Latitude,Temperature,Wind Speed,Humidity,Cloudiness,Hotel Name,Hotel Lat,Hotel Lng,Hotel City,Hotel Country
23,Laguna,-121.4238,38.4210,80.06,8.99,42,1,Holiday Inn Express & Suites Elk Grove Central...,38.4241,-121.399,elk grove,us
32,Atuona,-139.0333,-9.8000,78.89,9.46,72,12,Pension Temetiu Village,-9.80087,-139.032,atuona,pf
44,Vila Velha,-40.2925,-20.3297,82.35,8.05,69,0,Hotel Prainha,-20.3311,-40.2918,vila velha,br
112,Portel,-50.8211,-1.9356,81.54,3.62,76,15,Pousada Serve Bem,-1.93819,-50.8234,portel,br
139,Ukiah,-123.2078,39.1502,81.00,5.75,33,1,Economy Inn Ukiah,39.1477,-123.208,ukiah,us
239,Arraial do Cabo,-42.0278,-22.9661,80.46,8.05,74,0,Hotel Miramar,-22.9671,-42.0275,arraial do cabo,br
268,Khuzdar,66.6434,27.7384,81.01,6.42,12,0,Al Mustafa Hotel,27.8055,66.613,khuzdar,pk
297,Nuevo Progreso,-97.9500,23.1667,83.73,8.01,64,9,Villas Del Tordo,23.0468,-97.7862,nuevo progreso,mx
341,Kano,8.5167,12.0001,80.49,2.30,74,0,Royal Tropicana Hotel,12.0047,8.53644,kano,ng
358,Simdega,84.5167,22.6167,80.15,4.50,73,15,Mahabir Jain Bhawan,22.6112,84.5036,simdega,in


In [87]:
#create lists for making pins
hotelcity_list = vacation_df["Hotel City"].tolist()
hotelcountry_list = vacation_df["Hotel Country"].tolist()
hotelname_list = vacation_df["Hotel Name"].tolist()
#create locations for hotels
hotel_locations=vacation_df[['Hotel Lat','Hotel Lng']].astype(float)
#create list of info boxes

#create hotel location layer 
hotel_layer = gmaps.symbol_layer(
    hotel_locations, fill_color='rgba(150, 0, 150, 1)',
    stroke_color='rgba(150, 0, 150, 0.3)', scale=5,
    info_box_content=[
        f"Hotel Name: {hotelname}, City: {hotelcity}, Country: {hotelcountry}"
    for hotelname, hotelcity, hotelcountry
    in zip(hotelname_list, hotelcity_list, hotelcountry_list)
    ]
    #[f"Hotel Name: {hotelname}" for hotelname in hotelname_list]
    #info_box_content=[f", City: {hotelcity}" for hotelcity in hotelcity_list]
    #info_box_content=[f", Country: {hotelcountry}" for hotelcountry in hotelcountry_list]
                      )
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))